# **ETL de Google Maps**  
**Empezamos nuestro trabajo con la extracción de los datos de archivos json:**

Importamos librerias necesarias:

In [2]:
import pandas as pd
import json
import os

## **Extracción de Datos:**

El siguiente código crea diferentes dataframes por cada estado, leyendo cada archivo JSON de su respectivo estado.  
Guarda cada dataframe en una lista y estos luego son concatenados en un único dataframe.

In [2]:
# Función que retorna un dataframe, dependiendo de la ruta que varia en el ciclo for de más abajo
# Abre cada directorio diferente e itera sobre cada archivo JSON:
def cargar_reviews(ruta_estado):
    reviews_estado = []
    for directorio in os.listdir(ruta_estado):
        ruta_archivo = os.path.join(ruta_estado, directorio)
        with open(ruta_archivo, 'r') as archivo:
            for linea in archivo:
                review_data = json.loads(linea)
                reviews_estado.append(review_data)
    return pd.DataFrame(reviews_estado)

estados = ['california', 'florida', 'hawaii', 'nevada', 'new_york'] # Carpetas de estados

dfs = []    # Lista donde se guardan cada dataframe                       

# Iteramos por cada estado en la lista de estados:
for estado in estados:
    ruta_estado = f'../../Data/google_maps/reviews_estados/{estado}/' # Ruta de cada estados por iteración
    df_estado = cargar_reviews(ruta_estado)                   # Se llama a la función 'cargar_reviews'
    dfs.append(df_estado)                                     # Se añade el dataframe a la lista

**Concatenamos los dataframes en uno solo:**

In [3]:
df = pd.concat(dfs, ignore_index=True)

Eliminamos las columna innecesarias: 'pics' y 'resp'

In [4]:
drops = ["pics", "resp"]
df.drop(drops, axis=1, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11554347 entries, 0 to 11554346
Data columns (total 6 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   gmap_id  object
dtypes: int64(2), object(4)
memory usage: 528.9+ MB


### **Guardamos en un parquet los resultados:**    
**No queremos hacer devuelta todo el tiempo la lectura de json's (tarda demasiado)**

In [6]:
df.to_parquet("../../Data/data_preprocesada/reviews_estado_puro.parquet")

## **Transformación de Datos:**  

**Leemos el parquet guardado:**

In [3]:
df_reviews = pd.read_parquet("../../Data/data_preprocesada/reviews_estado_puro.parquet")

In [4]:
df_reviews.isna().sum()

user_id          0
name             0
time             0
rating           0
text       4800965
gmap_id          0
dtype: int64

Para poder filtrar las reviews de solo restaurantes, tenemos que traer ya los datos de estos mismos restaurantes, a traves del parquet:  
'restaurantes.parquet'. Para el primer paso traemos el archivo:

In [5]:
restaurantes = pd.read_parquet("../../Data/data_procesada/restaurantes.parquet")

Filtramos las reviews a traves de los restaurantes:

In [6]:
reviews_filtradas = df_reviews[df_reviews['gmap_id'].isin(restaurantes['gmap_id'])]

df_reviews_filtradas= pd.DataFrame(reviews_filtradas)

In [7]:
df_reviews_filtradas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1596014 entries, 0 to 11554346
Data columns (total 6 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1596014 non-null  object
 1   name     1596014 non-null  object
 2   time     1596014 non-null  int64 
 3   rating   1596014 non-null  int64 
 4   text     972481 non-null   object
 5   gmap_id  1596014 non-null  object
dtypes: int64(2), object(4)
memory usage: 85.2+ MB


In [8]:
df_reviews_filtradas.head(5)

,user_id,name,time,rating,text,gmap_id
0,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,112640357449611959087,David Han,1583643882296,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,117440349723823658676,Anthony Kim,1551938216355,5,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,100580770836123539210,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49


**Fusionaremos con merge() los Dataframes: 'restaurantes' y 'df_reviews_filtradas'**

Para más claridad cambiaremos los nombres de las columnas 'name' de ambos dataframes a nombres más especificos: user_name y restaurant_name.

In [9]:
df_reviews_filtradas.rename(columns={'name':'user_name'}, inplace=True)

restaurantes.rename(columns={'name':'restaurant_name'}, inplace=True)

In [10]:
# Fusionamos:
df_reviews_filtradas = pd.merge(df_reviews_filtradas, restaurantes[['gmap_id', 'restaurant_name', 'state', 'city']], on='gmap_id', how='inner')
df_reviews_filtradas.head(5)

,user_id,user_name,time,rating,text,gmap_id,restaurant_name,state,city
0,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,California,Los Angeles
1,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,California,Los Angeles
2,112640357449611959087,David Han,1583643882296,4,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,California,Los Angeles
3,117440349723823658676,Anthony Kim,1551938216355,5,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,California,Los Angeles
4,100580770836123539210,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,California,Los Angeles


In [11]:
df_reviews_filtradas.isna().sum()

user_id                 0
user_name               0
time                    0
rating                  0
text               623533
gmap_id                 0
restaurant_name         0
state                   0
city                    0
dtype: int64

Como observamos, los unicos nulos que posee nuestro Dataframe son el texto de las reviews, por lo tanto los rellenaremos con 'SD' (Sin Dato):

In [12]:
df_reviews_filtradas["text"] = df_reviews_filtradas["text"].fillna("SD")
df_reviews_filtradas.isna().sum()

user_id            0
user_name          0
time               0
rating             0
text               0
gmap_id            0
restaurant_name    0
state              0
city               0
dtype: int64

Reordenamos las columnas:

In [13]:
orden = ["user_id",	"user_name", "time", "rating", "text", "restaurant_name", "state", "city", "gmap_id"]

df_reviews_filtradas = df_reviews_filtradas[orden]
df_reviews_filtradas.head(2)

,user_id,user_name,time,rating,text,restaurant_name,state,city,gmap_id
0,108991152262655788985,Song Ro,1609909927056,5,Love there korean rice cake.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,111290322219796215751,Rafa Robles,1612849648663,5,Good very good,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49


**Convertimos a datetime la columna 'time'**  
Esta se encontraba en formato unix.

In [14]:
# Creamos una copia para trabajar con seguridad:
df_copy = df_reviews_filtradas.copy()

df_copy["time"] = pd.to_datetime(df_reviews_filtradas["time"], unit="ms")
df_copy.head()

,user_id,user_name,time,rating,text,restaurant_name,state,city,gmap_id
0,108991152262655788985,Song Ro,2021-01-06 05:12:07.056,5,Love there korean rice cake.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,111290322219796215751,Rafa Robles,2021-02-09 05:47:28.663,5,Good very good,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,112640357449611959087,David Han,2020-03-08 05:04:42.296,4,They make Korean traditional food very properly.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,117440349723823658676,Anthony Kim,2019-03-07 05:56:56.355,5,Short ribs are very delicious.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,100580770836123539210,Mario Marzouk,2017-05-16 05:01:41.933,5,Great food and prices the portions are large,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49


Ahora regularizamos los datos de la columna 'time' y ordenamos:

In [15]:
df_copy['year'] = df_copy['time'].dt.year
df_copy['month'] = df_copy['time'].dt.month
df_copy['day'] = df_copy['time'].dt.day
df_copy['hour'] = df_copy['time'].dt.strftime('%H:%M:%S.%f').str.rstrip('0')

orden_dos = ["user_id",	"user_name", "time", "year", "month", "day", "hour", "rating", "text", "restaurant_name", "state", "city", "gmap_id"]
df_copy = df_copy[orden_dos]
df_copy.head(3)

,user_id,user_name,time,year,month,day,hour,rating,text,restaurant_name,state,city,gmap_id
0,108991152262655788985,Song Ro,2021-01-06 05:12:07.056,2021,1,6,05:12:07.056,5,Love there korean rice cake.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,111290322219796215751,Rafa Robles,2021-02-09 05:47:28.663,2021,2,9,05:47:28.663,5,Good very good,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,112640357449611959087,David Han,2020-03-08 05:04:42.296,2020,3,8,05:04:42.296,4,They make Korean traditional food very properly.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49


Eliminamos la columna 'time':

In [16]:
df_copy.drop("time", axis=1, inplace=True)

df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596014 entries, 0 to 1596013
Data columns (total 12 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   user_id          1596014 non-null  object
 1   user_name        1596014 non-null  object
 2   year             1596014 non-null  int32 
 3   month            1596014 non-null  int32 
 4   day              1596014 non-null  int32 
 5   hour             1596014 non-null  object
 6   rating           1596014 non-null  int64 
 7   text             1596014 non-null  object
 8   restaurant_name  1596014 non-null  object
 9   state            1596014 non-null  object
 10  city             1596014 non-null  object
 11  gmap_id          1596014 non-null  object
dtypes: int32(3), int64(1), object(8)
memory usage: 127.9+ MB


**Verificación de Datos Duplicados:**

In [17]:
# Contar la cantidad de filas duplicadas
cantidad_duplicados = df_copy.duplicated().sum()
print("Cantidad de filas duplicadas:", cantidad_duplicados)

Cantidad de filas duplicadas: 67264


Observamos duplicados:

In [20]:
df_copy[df_copy.duplicated()].head(5)

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,gmap_id
169,101001321250592869991,Luis Briceno,2019,8,10,18:32:49.186,5,(Translated by Google) Very good place and ver...,La Superior Panaderia,California,Los Angeles,0x80c2c798ad41d859:0x8823cd86d2abd5b3
680,108991152262655788985,Song Ro,2021,1,6,05:12:07.056,5,Love there korean rice cake.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
681,111290322219796215751,Rafa Robles,2021,2,9,05:47:28.663,5,Good very good,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
682,112640357449611959087,David Han,2020,3,8,05:04:42.296,4,They make Korean traditional food very properly.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
683,117440349723823658676,Anthony Kim,2019,3,7,05:56:56.355,5,Short ribs are very delicious.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [24]:
# Vemos un caso en específico:
df_copy[df_copy["user_id"] == '117440349723823658676']

,user_id,user_name,year,month,day,hour,rating,text,restaurant_name,state,city,gmap_id
3,117440349723823658676,Anthony Kim,2019,3,7,05:56:56.355,5,Short ribs are very delicious.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49
683,117440349723823658676,Anthony Kim,2019,3,7,05:56:56.355,5,Short ribs are very delicious.,San Soo Dang,California,Los Angeles,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [27]:
# Eliminar las filas duplicadas
df_sin_duplicados = df_copy.drop_duplicates()

# Contar la cantidad de filas duplicadas nuevamente:
cantidad_duplicados = df_sin_duplicados.duplicated().sum()
print("Cantidad de filas duplicadas:", cantidad_duplicados)

Cantidad de filas duplicadas: 0


**Recuento y Promedio de Reseñas:**   
Los datos se agrupan por user_id y user_name para calcular el número total de reseñas por usuario y su calificación promedio.  
Este análisis centrado en el usuario puede revelar patrones en el comportamiento de las reseñas y el compromiso del usuario.

In [28]:
# Contar el número de reseñas por usuario
user_review_counts = df_sin_duplicados.groupby(['user_id', 'user_name']).size().reset_index(name='review_count')

# Calcular la calificación promedio por usuario
user_calif_promedio = df_sin_duplicados.groupby(['user_id', 'user_name'])['rating'].mean().reset_index()
user_calif_promedio.rename(columns={'rating': 'average_rating'}, inplace=True)

# Combinar los recuentos de reseñas y las calificaciones promedio por usuario en un solo DataFrame
df_user_review_counts_ord = pd.merge(user_review_counts, user_calif_promedio, on=['user_id', 'user_name'], how='inner')
df_user_review_counts_ord = df_user_review_counts_ord.sort_values(by='review_count', ascending=False)

# Mostrar las primeras filas del DataFrame ordenado
df_user_review_counts_ord.head()

,user_id,user_name,review_count,average_rating
261073,104819208193648646391,Gregor J. Rothfuss,101,3.950495
363428,106654503918907830147,The Corcoran Group,91,4.054945
822228,114955250538652050870,Javier Kohen,56,3.964286
829223,115082761597075271038,ej shortell,45,4.111111
530125,109673791694826464177,Jackie Gordon Singing Chef,44,3.863636


## **Carga de Datos:**  

Guardaremos nuestros dataframes en archivos parquet:

Datos de las reviews, Dataframe 'df_copy'. Lo usaremos para análisis.

In [29]:
df_copy.to_parquet("../../Data/data_procesada/reviews_google_maps.parquet", index=False)

Datos de los ususariois, Dataframe 'df_user_review_counts_ord'. Lo usaremos para análisis.

In [30]:
df_user_review_counts_ord.to_parquet("../../Data/data_procesada/users_google_maps.parquet", index=False)